In the NCLT dataset the LiDAR measurements are done with the Velodyne HDL-32E. For each day we have two kinds of data:
- velodyne_hits.bin: binary file with all raw velodyne scans of the session with
    - timestamp
    - x,y,z - coordinates
    - intensity
    laser id
- velodyne_snyc: contains one binary file for each timestamp with movement and time sync 360° scans

We need the files from velodyne_sync

In [1]:
import numpy as np
from pathlib import Path
from sklearn.metrics import precision_recall_curve, average_precision_score, roc_curve, auc

from src.utils import calculate_m2dp_descriptor
from src.data_loader import save_timestamps, load_place_recognition_data
from src.similarity_measure import retrieve_candidates, retrieve_loop_pos, visualize_trajectory, get_gt_trajectory

SCAN_FOLDER = "data/velodyne"
DESCRIPTORS_FOLDER = "m2dp_descriptors"
GT_POSE_FOLDER = "data/gt_pose"
GT_TIMESTAMP_FOLDER = "data/gt_timestamps"

### Saving timestamps of LiDAR scans

In [2]:
base = Path(SCAN_FOLDER)
max_depth = 3

for p in base.rglob("*"):
    if p.is_dir():
        depth = len(p.relative_to(base).parts)
        if depth == max_depth:
            save_timestamps(p, TIMESTAMP_FOLDER)

Writing timestamps in data/timestamps/2012-01-15.csv: 33612it [00:00, 268971.93it/s]


CSV saved.



Writing timestamps in data/timestamps/2012-01-08.csv: 28127it [00:00, 270355.62it/s]


CSV saved.



Writing timestamps in data/timestamps/2012-01-22.csv: 26145it [00:00, 275877.19it/s]


CSV saved.



### M2DP Descriptor to capture geometric structure of LiDAR scans

In [6]:
base = Path(SCAN_FOLDER)
max_depth = 3

for p in base.rglob("*"):
    if p.is_dir():
        depth = len(p.relative_to(base).parts)
        if depth == max_depth:
            calculate_m2dp_descriptor(folder=p, output_dir="m2dp_descriptors", save=True, parallelize=True)

Found 33612 scans in data/velodyne/2012-01-15_vel/2012-01-15/velodyne_sync.
Parallelizing process with multi-threading.


Processing files: 100%|██████████| 33612/33612 [35:03<00:00, 15.98it/s]  


Done. Signature vector matrix shape: (33612, 32)
Descriptors saved in m2dp_descriptors/m2dp_2012-01-15.npy
Timestamps saved in m2dp_descriptors/timestamps_2012-01-15.npy
Found 28127 scans in data/velodyne/2012-01-08_vel/2012-01-08/velodyne_sync.
Parallelizing process with multi-threading.


Processing files: 100%|██████████| 28127/28127 [28:24<00:00, 16.50it/s]  


Done. Signature vector matrix shape: (28127, 32)
Descriptors saved in m2dp_descriptors/m2dp_2012-01-08.npy
Timestamps saved in m2dp_descriptors/timestamps_2012-01-08.npy
Found 26145 scans in data/velodyne/2012-01-22_vel/2012-01-22/velodyne_sync.
Parallelizing process with multi-threading.


Processing files: 100%|██████████| 26145/26145 [26:42<00:00, 16.32it/s]  


Done. Signature vector matrix shape: (26145, 32)
Descriptors saved in m2dp_descriptors/m2dp_2012-01-22.npy
Timestamps saved in m2dp_descriptors/timestamps_2012-01-22.npy


### Histogram-Based Similarity Measure for Loop Closure Detection in 3D LiDAR Data

In [29]:
def run_evaluations(candidates_loop_pos, y_scores, gt_loop_pos):
    N = len(candidates_loop_pos)
    y_true = [None] * N
    for i in range(N):
        temp = candidates_loop_pos[i]
        exists = any(
            np.array_equal(t[0], temp[0]) and np.array_equal(t[1], temp[1])
            for t in gt_loop_pos
        )

        if exists:
            y_true[i] = 1
        else:
            y_true[i] = 0

    precision, recall, pr_thresholds = precision_recall_curve(y_true, y_scores)
    ap_score = average_precision_score(y_true, y_scores)
    
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)
    
    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-9)
    best_f1 = np.max(f1_scores)

In [ ]:
tau = 0.025 # 0.2 gibt gefühlt nix, 0.3 gibt zu viel
window = 200
topk = 1

r = 3.0
time_window = 30

for p in Path(DESCRIPTORS_FOLDER+"/descriptors").iterdir():
    day = p.name.split("_")[1].removesuffix(".npy")    
    descriptors, timestamps, gt, gt_timestamps = load_place_recognition_data(p, DESCRIPTORS_FOLDER, GT_POSE_FOLDER, GT_TIMESTAMP_FOLDER)
    
    # calculate loop candidates, its position based on gt timestamps and gt loops

    loop_candidates = retrieve_candidates(descriptors, window, tau, topk)
    candidates_loop_pos, gt_loop_pos = retrieve_loop_pos(loop_candidates, timestamps, gt, time_window, r)

    # visualize trajectory with loop positions
    trajectory = get_gt_trajectory(gt, gt_timestamps)
    visualize_trajectory(trajectory[0], candidates_loop_pos, gt_loop_pos, day)

    # evaluate
    scores = loop_candidates[:, 2]
    run_evaluations(candidates_loop_pos, scores, gt_loop_pos)
    
    break
    print()

In [31]:
run_evaluations(candidates_loop_pos, scores, gt_loop_pos)